## Installation

In [1]:
! pip install wyn-voice pyautogen pydub

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.2/345.2 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00


In [15]:
! pip install wyn-wav2lip==0.2.3

In [16]:
from wyn_wav2lip.wav2lip import *

In [19]:
# Initialize Wav2Lip
wav2lip = Wav2Lip()

# Setup environment (ensure that you have GPU enabled)
wav2lip.setup()

## Imports

In [3]:
from wyn_voice.chat import *
import tensorflow as tf
from autogen import ConversableAgent
from pydub import AudioSegment

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [4]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [5]:
from wyn_wav2lip.easy_functions import *

In [6]:
from pathlib import Path
from autogen.coding import CodeBlock, LocalCommandLineCodeExecutor

work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)
executor = LocalCommandLineCodeExecutor(work_dir=work_dir)

# Create an agent with code executor configuration.
code_executor_agent = ConversableAgent(
    "code_executor_agent",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the local command line code executor.
    human_input_mode="NEVER",  # Always take human input for this agent for safety.
)

## Configure and Run

In [7]:
# @title Choose a bot

# Reinitiate bot
# Example usage
assistant_v1 = ChatBot(
    api_key=OPENAI_API_KEY,
    protocol="""
    You are a helpful assistant.

    If user asks you python related question, you will ask question back and clarify it.
    If all makes sense, you will write the python code for user.
    If the python code is written from before, ask user whether they want to execute it.
    """
)
assistant_v2 = ChatBot(
    api_key=OPENAI_API_KEY,
    protocol="""
    You are a helpful assistant. You are an expert to recommend life style choices.
    """
)
assistant_v3 = ChatBot(
    api_key=OPENAI_API_KEY,
    protocol="""
    You are a therapist.

    You are an expert at guiding people through mental health stresses and work situations.
    You are a good listener and you take the patients (user) complain and try to understand it.
    When user ask, you may suggest some life style choices to the user based on the user's complain.
    """
)
assistant_v4 = ChatBot(
    api_key=OPENAI_API_KEY,
    protocol="""
    You are a McDonald drivethrough service assistant.

    Here's a menu for the day:

    | Item                       | Price  | Calories |
    |----------------------------|--------|----------|
    | Big Mac                    | $5.99  | 550      |
    | Quarter Pounder with Cheese| $6.49  | 750      |
    | McChicken                  | $3.29  | 400      |
    | Filet-O-Fish               | $4.99  | 390      |
    | French Fries (Medium)      | $2.99  | 340      |
    | Chicken McNuggets (10 pcs) | $5.49  | 480      |
    | McDouble                   | $2.99  | 390      |
    | Egg McMuffin               | $3.99  | 300      |
    | Sausage Biscuit            | $2.79  | 460      |
    | Apple Pie                  | $1.49  | 240      |

    You are designed to serve the cutomer with the best price and calories combination.

    User will ask you some questions and price and calories.

    Ask user if they are ready to confirm the order.

    If user is ready to confirm the order, write a python code for the order. The python code saves the order in a csv file.
    If user is not ready to confirm the order, ask user if there's anything to add.

    """
)

# Choose assistant
value = "Therapist" # @param ["Coding Helper", "Lifestyle Helper", "Therapist", "Drivethrough"]

def set_assistant():
    if value == "Coding Helper":
        assistant = assistant_v1
    elif value == "Lifestyle Helper":
        assistant = assistant_v2
    elif value == "Therapist":
        assistant = assistant_v3
    else:
        assistant = assistant_v4
    audio_processor = AudioProcessor(bot=assistant)

    return assistant, audio_processor

assistant, audio_processor = set_assistant()

### Talk to JARVIS

In [ ]:
# @title Start v1!

greetings = False
if greetings:
    # Greetings
    response_short = "Hi, welcome to our store! How may I help you?"
    print("🤖 Bot: ", response_short)

    # Audio output
    output_file_path = audio_processor.text_to_voice(response_short)
    audio = AudioSegment.from_file(output_file_path)
    audio_length = len(audio) / 1000.0 + 1
    print("⏳ Audio lag: ", audio_length)

prompt = ""
response = ""
reply = ""
audio_length = 0
while "exit" not in prompt.lower():
    # User
    prompt = audio_processor.voice_to_text(sec=4)
    print("🤔 User: ", prompt)

    # Command [conditional]
    if "execute" in prompt.lower():
        message_with_code_block = f"""This is a message with code block.
        {response}
        """

        # Generate a reply for the given code.
        reply = code_executor_agent.generate_reply(messages=[{"role": "user", "content": message_with_code_block}])
        print("💻 Executing....")
        print(reply)

        # Bot
        try:
            response = assistant.generate_response(
                f"""User has provided the prompt: {prompt}
                And previous we have an answer: {response}

                Now that the code finished executing. Here's the result: {reply}.

                Summarize the result and ask for another question or task. """
            )
            print("🤖 Bot: ", response)
        except:
            assistant, audio_processor = set_assistant()
            response = assistant.generate_response(
                f"""User has provided the prompt: {prompt}
                And previous we have an answer: {response}

                Now that the code finished executing. Here's the result: {reply}.

                Summarize the result and ask for another question or task. """
            )
            print("🤖 Bot: ", response)

    else:
        # Bot
        try:
            response = assistant.generate_response(prompt)
            print("🤖 Bot: ", response)
        except:
            assistant, audio_processor = set_assistant()
            response = assistant.generate_response(prompt)
            print("🤖 Bot: ", response)

    # Audio
    if "\n\n```python" in response.lower() or len(response) > 500:
        response_short = assistant.generate_response(f"Summarize the following in 1-2 sentences: {response}")
    else:
        response_short = response

    # Audio output
    output_file_path = audio_processor.text_to_voice(response_short)
    # audio = AudioSegment.from_file(output_file_path)
    # audio_length = len(audio) / 1000.0 + 1
    # print("⏳ Audio lag: ", audio_length)

    # Video output
    with tf.device('/device:GPU:0'):
        print("Rendering video...")
        wav2lip.run(video_file="/content/yiqiao3.jpg", vocal_file=f"/content/Easy-Wav2Lip/{output_file_path}")

    # Wait from last round
    # time.sleep(audio_length)

In [32]:
# @title Start v2!

prompt = ""
response = ""
reply = ""
audio_length = 0

# User
prompt = audio_processor.voice_to_text(sec=5)
print("🤔 User: ", prompt)

# Bot
response = assistant.generate_response(prompt)
print("🤖 Bot: ", response)

# Audio output
output_sound_file_path = audio_processor.text_to_voice(response, True)

Recording audio...


<IPython.core.display.Javascript object>

Audio saved!
Transcribing audio from: audio.wav
🤔 User:  Write a poem about the United States.
🤖 Bot:  In the land of stars and stripes so bold,
Where dreams are crafted, and stories told.
From coast to coast, a nation stands strong,
United in diversity, where all belong.

In bustling cities, skyscrapers rise high,
A melting pot of cultures under the sky.
From amber waves of grain to mountains grand,
The beauty of the land stretches vast and grand.

Freedom's beacon shines bright and clear,
Opportunity and hope always near.
Through trials and triumphs, the nation thrives,
In the hearts of its people, freedom survives.

Oh, United States, land of the free,
May your spirit of unity forever be.
In the land where dreams take flight,
America shines with radiant light.


In [33]:
print(output_sound_file_path)
wav2lip.run(
        video_file="/content/yiqiao3.jpg",
        vocal_file="/content/Easy-Wav2Lip/output.mp3")

# Call the function with the path to the video file
autoplay_video('/content/Easy-Wav2Lip/temp/output.mp4')

output.mp3
Output video not found at: output.mp4
